# CURLY VS STRAIGHT

- model class
- Dataset class
- Training loop

In [1]:
# get data
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

--2025-12-02 10:57:16--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T11%3A39%3A18Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T10%3A39%3A00Z&ske=2025-12-02T11%3A39%3A18Z&sks=b&skv=2018-11-09&sig=8jPH0iGduaFi9CtXKz2o0InNpzjNc0P1piJKN4r9%2BmQ%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDY3NDgzNiwibmJmIjoxNzY0NjczMDM2LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

In [2]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# check pytorch version
torch.__version__

'2.9.0+cu126'

In [9]:
# Develop Model
import torch
import torch.nn as nn
import torch.optim as optim

INPUT_SIZE = (3, 200, 200)

class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()

        # conv layer
        # Input: (3, 200, 200) -> Output: (32, 198, 198)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, stride=1)
        self.relu = nn.ReLU()

        # pool layer
        # Input: (32, 198, 198) -> Output: (32, 99, 99)
        self.pool = nn.MaxPool2d(kernel_size=(2,2))

        # linear 1
        # Flatten size = 32 channels * 99 height * 99 width = 313,632
        self.fc1 = nn.Linear(in_features=32 * 99 * 99, out_features=64)

        # 4. Output Layer
        self.fc2 = nn.Linear(in_features=64, out_features=1)

        # Binary Classification Activation
        # self.sigmoid = nn.Sigmoid()
        # not using sigmoid anymore so i can use BCEWithLogitsLoss

    def forward(self, x):
        # Convolution -> ReLU -> MaxPool -> flatten -> fc1 -> relu -> output (sigmoid)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        # x.size(0) preserves the batch size
        x = x.view(x.size(0), -1)


        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        # x = self.sigmoid(x)

        return x

In [10]:
# Instantiate the model and put on appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BinaryClassifier()

model.to(device);

# Define the Optimizer and loss func
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss() # as we're using sigmoid

In [11]:
print(model)

BinaryClassifier(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=313632, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [12]:
# Q2
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------


In [13]:
# resize + imagenet normalization
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [14]:
# create Dataset class
import os
from torch.utils.data import Dataset
from PIL import Image

class HairDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        # initialize with dir, transforms, imgpaths, labels, classes and classmap
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir)) # folder-names are classes
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        # loop through each class and save all img paths + label (class)
        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    # number of imgs
    def __len__(self):
        return len(self.image_paths)

    # yields image-label pairs
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [16]:
from torch.utils.data import DataLoader

train_dataset = HairDataset(
    data_dir='/content/data/train',
    transform=train_transforms
)

validation_dataset = HairDataset(
    data_dir='/content/data/test',
    transform=val_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

In [18]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6511, Acc: 0.6105, Val Loss: 0.7584, Val Acc: 0.6169
Epoch 2/10, Loss: 0.6398, Acc: 0.6592, Val Loss: 0.6111, Val Acc: 0.6468
Epoch 3/10, Loss: 0.5551, Acc: 0.7041, Val Loss: 0.7363, Val Acc: 0.5970
Epoch 4/10, Loss: 0.5516, Acc: 0.6841, Val Loss: 0.5790, Val Acc: 0.6517
Epoch 5/10, Loss: 0.4791, Acc: 0.7628, Val Loss: 0.6026, Val Acc: 0.6766
Epoch 6/10, Loss: 0.4158, Acc: 0.7878, Val Loss: 0.7369, Val Acc: 0.6269
Epoch 7/10, Loss: 0.5344, Acc: 0.7316, Val Loss: 0.6704, Val Acc: 0.6418
Epoch 8/10, Loss: 0.4029, Acc: 0.8065, Val Loss: 0.6103, Val Acc: 0.6667
Epoch 9/10, Loss: 0.3768, Acc: 0.8190, Val Loss: 0.6191, Val Acc: 0.6667
Epoch 10/10, Loss: 0.3504, Acc: 0.8140, Val Loss: 0.7743, Val Acc: 0.6716


In [24]:
# Q3 - median training accuracy
import numpy as np
np.median(history['acc'])

np.float64(0.747191011235955)

In [25]:
#Q4 - std training loss
np.std(history['loss'])

np.float64(0.1017799748268336)

In [26]:
# more augmentation for train dataset
train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = HairDataset(
    data_dir='/content/data/train',
    transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

In [27]:
# train for 10 more epochs
num_epochs = 10
new_history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    new_history['loss'].append(epoch_loss)
    new_history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    new_history['val_loss'].append(val_epoch_loss)
    new_history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6044, Acc: 0.6866, Val Loss: 0.5975, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5800, Acc: 0.7004, Val Loss: 0.7135, Val Acc: 0.6517
Epoch 3/10, Loss: 0.5635, Acc: 0.7066, Val Loss: 0.5538, Val Acc: 0.7065
Epoch 4/10, Loss: 0.5730, Acc: 0.6991, Val Loss: 0.5991, Val Acc: 0.6716
Epoch 5/10, Loss: 0.5310, Acc: 0.7104, Val Loss: 0.5680, Val Acc: 0.6965
Epoch 6/10, Loss: 0.4940, Acc: 0.7428, Val Loss: 0.9832, Val Acc: 0.5473
Epoch 7/10, Loss: 0.6356, Acc: 0.6654, Val Loss: 0.7053, Val Acc: 0.5821
Epoch 8/10, Loss: 0.5455, Acc: 0.7104, Val Loss: 0.5573, Val Acc: 0.6965
Epoch 9/10, Loss: 0.5167, Acc: 0.7378, Val Loss: 0.9860, Val Acc: 0.5721
Epoch 10/10, Loss: 0.5294, Acc: 0.7341, Val Loss: 0.5708, Val Acc: 0.7264


In [28]:
# Q5 - mean test loss
np.mean(new_history['val_loss'])

np.float64(0.6834502427268829)

In [30]:
# Q6 - avg test accuracy for epochs 6-10
np.mean(new_history["val_acc"][5:])

np.float64(0.6248756218905472)